## Mini Project III
#### November 16, 2021
Ali Bahrami

### Topics
This mini project is dedicated to following topics:
- Data Wrangling
- Data Visualization
- Data Preparation and Feature Engineering
- Dimensionality Reduction
- Unsupervised Learning

### Data
We will be using old data about different financial transactions. You can download the data from [here](https://drive.google.com/file/d/1zAjnf936aHkwVCq_BmA47p4lpRjyRzMf/view?usp=sharing). The data contains following tables:

- twm_customer - information about customers
- twm_accounts - information about accounts
- twm_checking_accounts - information about checking accounts (subset of twm_accounts)
- twm_credit_accounts - information about checking accounts (subset of twm_accounts)
- twm_savings_accounts - information about checking accounts (subset of twm_accounts)
- twm_transactions - information about financial transactions
- twm_savings_tran - information about savings transactions (subset of twm_transactions)
- twm_checking_tran - information about savings transactions (subset of twm_transactions)
- twm_credit_tran - information about credit checking (subset of twm_transactions)

In [2]:
# Import libraries
import pandas as pd

from sqlalchemy import create_engine

## Read data and create database

In [5]:
# Read files

df_customer = pd.read_csv('data/twm_customer.csv', delimiter=';')
df_accounts = pd.read_csv('data/twm_accounts.csv', delimiter=';')
df_transactions = pd.read_csv('data/twm_transactions.csv', delimiter=';')

df_checking_acct = pd.read_csv('data/twm_checking_acct.csv', delimiter=';')
df_checking_tran = pd.read_csv('data/twm_checking_tran.csv', delimiter=';')

df_savings_acct = pd.read_csv('data/twm_savings_acct.csv', delimiter=';')
df_savings_tran = pd.read_csv('data/twm_savings_tran.csv', delimiter=';')

df_credit_acct = pd.read_csv('data/twm_credit_acct.csv', delimiter=';')
df_credit_tran = pd.read_csv('data/twm_credit_tran.csv', delimiter=';')

In [6]:
db_connection = create_engine('sqlite:///data/banking.db')
conn = db_connection.connect()

In [7]:
df_customer.to_sql('customer', conn, if_exists='replace', index=False)
df_accounts.to_sql('accounts', conn, if_exists='replace', index=False)
df_transactions.to_sql('transcations', conn, if_exists='replace', index=False)

df_savings_tran.to_sql('savings_tran', conn, if_exists='replace', index=False)
df_savings_acct.to_sql('savings_acct', conn, if_exists='replace', index=False)

df_credit_tran.to_sql('credit_tran', conn, if_exists='replace', index=False)
df_credit_acct.to_sql('credit_acct', conn, if_exists='replace', index=False)

df_checking_tran.to_sql('checking_tran', conn, if_exists='replace', index=False)
df_checking_acct.to_sql('checking_acct', conn, if_exists='replace', index=False)

## Calculate Credit_limit

In [8]:
query_limit = """SELECT cust_id, credit_limit FROM credit_acct;"""
df_limit = pd.read_sql(query_limit, conn)
df_limit

,cust_id,credit_limit
0,1363160,1000
1,1362487,3000
2,1362548,6700
3,1362752,2400
4,1363017,1000
...,...,...
463,1363364,3000
464,1363467,1500
465,1363263,3000
466,1362569,9200


## Calculate Net Worth of Each Customer

In [9]:
query = """
SELECT customer.cust_id, accounts.acct_type, accounts.ending_balance

FROM customer

JOIN accounts ON customer.cust_id = accounts.cust_id
"""

In [10]:
df_accounts = pd.read_sql(query, conn)
df_accounts.sample(5)

,cust_id,acct_type,ending_balance
929,1362623,CK,16.18
858,1363237,CC,0.00
1292,1362735,SV,288.87
897,1362930,SV,800.42
145,1362727,CC,0.00


In [11]:
checking = df_accounts[df_accounts['acct_type'].str.contains('CK')]
checking = checking.rename(columns={"acct_type": "checking_acct", "ending_balance": "checking_balance"})
checking.head()

savings = df_accounts[df_accounts['acct_type'].str.contains('SV')]
savings = savings.rename(columns={"acct_type": "savings_acct", "ending_balance": "savings_balance"})
savings.head()

credit = df_accounts[df_accounts['acct_type'].str.contains('CC')]
credit = credit.rename(columns={"acct_type": "credit_acct", "ending_balance": "credit_balance"})
credit.head()

,cust_id,credit_acct,credit_balance
0,1362487,CC,1548.23
3,1363160,CC,286.69
5,1362752,CC,0.00
8,1362548,CC,68.68
11,1363017,CC,1000.00


In [21]:
unique_cust_id = df_accounts.cust_id.unique()
df_net_worth = pd.DataFrame(unique_cust_id, columns=['cust_id'])
df_net_worth = df_net_worth.merge(checking, on='cust_id', how='outer')
df_net_worth = df_net_worth.merge(savings, on='cust_id', how='outer')
df_net_worth = df_net_worth.merge(credit, on='cust_id', how='outer')
df_net_worth = df_net_worth.merge(df_limit, on='cust_id', how='outer')
df_net_worth = df_net_worth.fillna(value=0)

In [29]:
df_net_worth['net_worth'] = df_net_worth['savings_balance'] + df_net_worth['checking_balance'] - df_net_worth['credit_balance']
df_net_worth['credit_spent_(%)'] = (df_net_worth['credit_balance'] / df_net_worth['credit_limit'])*100


In [30]:
df_net_worth = df_net_worth[['cust_id','checking_balance', 'savings_balance', 'credit_balance','credit_limit', 'credit_spent_(%)' ,'net_worth']]
df_net_worth = df_net_worth.fillna(value=0)
df_net_worth

,cust_id,checking_balance,savings_balance,credit_balance,credit_limit,credit_spent_(%),net_worth
0,1362487,1401.21,12.89,1548.23,3000.0,51.607667,-134.13
1,1363160,0.00,122.54,286.69,1000.0,28.669000,-164.15
2,1362752,3.76,609.33,0.00,2400.0,0.000000,613.09
3,1362548,569.65,254.02,68.68,6700.0,1.025075,754.99
4,1363017,147.15,2901.35,1000.00,1000.0,100.000000,2048.50
...,...,...,...,...,...,...,...
660,1363324,2325.37,0.00,0.00,0.0,0.000000,2325.37
661,1362895,0.00,96.25,1300.00,1300.0,100.000000,-1203.75
662,1362569,46.48,0.00,9200.00,9200.0,100.000000,-9153.52
663,1363364,296.28,0.00,1785.97,3000.0,59.532333,-1489.69


## Get interests paid on credit for each customer

In [31]:
query_interest = """
SELECT customer.cust_id, customer.income, customer.age, 
customer.nbr_children, customer.gender, customer.marital_status,
customer.city_name, customer.state_code, SUM(credit_tran.interest_amt) AS credit_interests

FROM customer

JOIN credit_tran ON customer.cust_id = credit_tran.cust_id

GROUP BY customer.cust_id;
"""

In [32]:
df_credit_interest = pd.read_sql(query_interest, conn)
df_credit_interest

,cust_id,income,age,nbr_children,gender,marital_status,city_name,state_code,credit_interests
0,1362480,50890,33,2,M,2,New York City,NY,361.26
1,1362484,10053,42,0,F,1,Denver,CO,14.81
2,1362485,22690,25,0,F,1,New York City,NY,56.93
3,1362487,6605,71,0,M,2,Los Angeles,CA,13.63
4,1362489,55888,35,2,F,3,Nashville,TN,248.77
...,...,...,...,...,...,...,...,...,...
452,1363483,9271,28,0,M,2,Washington,DC,79.61
453,1363484,10244,52,0,F,1,Minneapolis,MN,12.21
454,1363485,19863,48,0,F,1,Seattle,WA,41.27
455,1363486,39942,41,5,F,4,Honolulu,HI,0.00


## Get all transactions for each customer

In [33]:
query_checking = """SELECT cust_id, tran_id AS checking_trans FROM checking_tran GROUP BY cust_id;"""
df_checking = pd.read_sql(query_checking, conn)

In [34]:
query_savings = """ SELECT cust_id, tran_id AS savings_trans FROM savings_tran GROUP BY cust_id;"""
df_savings = pd.read_sql(query_savings, conn)


In [35]:
query_credit = """SELECT cust_id, tran_id AS credit_trans FROM credit_tran GROUP BY cust_id;"""
df_credit = pd.read_sql(query_credit, conn)

In [36]:
df_trans = pd.merge(left=df_credit, right=df_checking, left_on='cust_id', right_on='cust_id', how='left')
df_trans = pd.merge(left=df_trans, right=df_savings, left_on='cust_id', right_on='cust_id', how='left')
df_trans = df_trans.fillna(0)
df_trans['total_trans'] = df_trans['checking_trans'] + df_trans['savings_trans'] + df_trans['credit_trans']
df_trans = df_trans.astype(int)
df_trans

,cust_id,credit_trans,checking_trans,savings_trans,total_trans
0,1362480,14,56,14,84
1,1362484,18,18,18,54
2,1362485,47,47,0,94
3,1362487,5,5,5,15
4,1362489,20,84,20,124
...,...,...,...,...,...
452,1363483,31,31,0,62
453,1363484,39,91,0,130
454,1363485,9,0,9,18
455,1363486,17,0,0,17


## Merge and Export

In [37]:
df_final = df_credit_interest
df_final = pd.merge(left=df_final, right=df_trans, left_on='cust_id', right_on='cust_id', how='left')
df_final = pd.merge(left=df_final, right=df_net_worth, left_on='cust_id', right_on='cust_id', how='left')
df_final

,cust_id,income,age,nbr_children,gender,marital_status,city_name,state_code,credit_interests,credit_trans,checking_trans,savings_trans,total_trans,checking_balance,savings_balance,credit_balance,credit_limit,credit_spent_(%),net_worth
0,1362480,50890,33,2,M,2,New York City,NY,361.26,14,56,14,84,86.86,1187.97,20.00,2500.0,0.800000,1254.83
1,1362484,10053,42,0,F,1,Denver,CO,14.81,18,18,18,54,95.32,2122.17,369.72,3000.0,12.324000,1847.77
2,1362485,22690,25,0,F,1,New York City,NY,56.93,47,47,0,94,45.49,0.00,1100.00,1100.0,100.000000,-1054.51
3,1362487,6605,71,0,M,2,Los Angeles,CA,13.63,5,5,5,15,1401.21,12.89,1548.23,3000.0,51.607667,-134.13
4,1362489,55888,35,2,F,3,Nashville,TN,248.77,20,84,20,124,125.43,373.65,20.00,2800.0,0.714286,479.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1363483,9271,28,0,M,2,Washington,DC,79.61,31,31,0,62,403.75,0.00,1000.00,1000.0,100.000000,-596.25
453,1363484,10244,52,0,F,1,Minneapolis,MN,12.21,39,91,0,130,233.06,0.00,408.08,1000.0,40.808000,-175.02
454,1363485,19863,48,0,F,1,Seattle,WA,41.27,9,0,9,18,0.00,157.76,1000.00,5000.0,20.000000,-842.24
455,1363486,39942,41,5,F,4,Honolulu,HI,0.00,17,0,0,17,0.00,0.00,0.00,2000.0,0.000000,0.00


In [38]:
# Export
df_final.to_csv('data/final_data.csv', sep=',', index=False)